prepare documents.

In [2]:
import json
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [ ]:
docs_raw[0]

formatting the docs_raw in order so that we could feed that directly to elastic search, 
currently we have 2 different level on the docs_raw

In [9]:
documents = []
for courses in docs_raw:
    for doc in courses['documents']:
        doc['course'] = courses['course']
        documents.append(doc)

In [10]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [15]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# The sentences to encode
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

(3, 384)


In [16]:
model = SentenceTransformer("all-mpnet-base-v2")

In [18]:
dense_vector = model.encode('this is a simple sentence')

In [19]:
# create dense vector using pre-trained model.

operation = []
for doc in documents:
    #tranforming the text into an embedding using the model
    doc['text_vector'] = model.encode(doc['text']).tolist()
    operation.append(doc)

In [22]:
len(operation[0]['text_vector'])

768

In [23]:
from elasticsearch import Elasticsearch

In [24]:
es_client = Elasticsearch('http://localhost:9200')

In [27]:
print(es_client.info())

{'name': 'f6f7b863cbf2', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Ix9Gw4vKQNKj_jHEP6w8WQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


Mapping and index for elastic search

In [28]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [29]:
index_name = 'course-quesiton'

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-quesiton'})

In [30]:
for doc in operation:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)
    

In [31]:
search_term = 'window or mac?'
vector_search_term = model.encode(search_term)

In [46]:
query = {
    "field" : "text_vector",
    "query_vector": vector_search_term,
    "k" : 5,
    "num_candidates": 10000
}

In [47]:
result = es_client.search(index=index_name, knn=query, source=['text', 'section', 'question', 'course'])

In [48]:
result['hits']['hits']

[{'_index': 'course-quesiton',
  '_id': 'ywT_sZABAj9vCZ-bxloI',
  '_score': 0.7011841,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-quesiton',
  '_id': 'nwT_sZABAj9vCZ-b7Fwh',
  '_score': 0.61774516,
  '_source': {'question': "The answer I get for one of the homework questions doesn't match any of the options. What should I do?",
   'course': 'machine-learning-zoomcamp',
   'section': '2. Machine Learning for Regression',
   'text': 'That’s normal. We all have different environments: our computers have different versions of OS and different versions of libraries — even different versions of Python.\nIf it’s the case, just select the option that’s closest to your answer'}},
 {'_index': 'course-quesiton',
  '_id'